1. Import
2. Clean
3. Transform - OneHot, standardscaling. Save .pkl.
4. Build, Compile ANN
5. Hyperparameter tuning
6. Save model .h5.

In [43]:
# Importing and pre-processing the data
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle
import os

# ANN Implementation
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Hyperparameter Tuning
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

## Importing the data

In [44]:
df = pd.read_csv("Datasets/Churn_Modelling.csv")

In [45]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Dropping Unwanted Columns

#### We don't need the first 3 columns for classification - RowNumber, CustomerId and Surname

In [46]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [47]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Checking for null values

In [48]:
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

## Checking Datatypes

In [49]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

## Train, Test Split

In [50]:
# Independent features
X = df.drop(['Exited'], axis=1)

# Dependent Feature
y = df['Exited']

In [51]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [52]:
y[:5]

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [53]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
X_train_raw.shape

(8000, 10)

In [55]:
X_test_raw.shape

(2000, 10)

## Preprocessing

In [56]:
# Categorical Columns
cat_cols = X.select_dtypes(include='object').columns.tolist()

# Numerical Columns
num_cols = X.select_dtypes(include=np.number).columns.tolist()

# We do not need to do StandardScaling on binary columns 'HasCrCard' and 'IsActiveMember'
num_cols.remove('HasCrCard')
num_cols.remove('IsActiveMember')

In [57]:
print(*cat_cols)
print(*num_cols)

Geography Gender
CreditScore Age Tenure Balance NumOfProducts EstimatedSalary


In [58]:
# Defining the preprocessor
preprocessor = ColumnTransformer(transformers = [('num', StandardScaler(), num_cols),
                                                ('cat', OneHotEncoder(), cat_cols)],
                                remainder='drop')

In [59]:
X_train = preprocessor.fit_transform(X_train_raw)
X_test = preprocessor.transform(X_test_raw)

In [60]:
X_train[:5]

array([[ 0.35649971, -0.6557859 ,  0.34567966, -1.21847056,  0.80843615,
         1.36766974,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.20389777,  0.29493847, -0.3483691 ,  0.69683765,  0.80843615,
         1.6612541 ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ],
       [-0.96147213, -1.41636539, -0.69539349,  0.61862909, -0.91668767,
        -0.25280688,  0.        ,  0.        ,  1.        ,  0.        ,
         1.        ],
       [-0.94071667, -1.13114808,  1.38675281,  0.95321202, -0.91668767,
         0.91539272,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [-1.39733684,  1.62595257,  1.38675281,  1.05744869, -0.91668767,
        -1.05960019,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ]])

In [61]:
X_test[:5]

array([[-0.57749609, -0.6557859 , -0.69539349,  0.32993735,  0.80843615,
        -1.01960511,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ],
       [-0.29729735,  0.3900109 , -1.38944225, -1.21847056,  0.80843615,
         0.79888291,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.52560743,  0.48508334, -0.3483691 , -1.21847056,  0.80843615,
        -0.72797953,  0.        ,  0.        ,  1.        ,  1.        ,
         0.        ],
       [-1.51149188,  1.91116988,  1.03972843,  0.68927246,  0.80843615,
         1.22138664,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ],
       [-0.9510944 , -1.13114808,  0.69270405,  0.78283876, -0.91668767,
         0.24756011,  0.        ,  0.        ,  1.        ,  1.        ,
         0.        ]])

#### Saving the preprocessor

In [62]:
with open('preprocessor.pkl', 'wb') as file:
    pickle.dump(preprocessor, file)

## Building the ANN

#### Defining the model

In [63]:
model_1 = Sequential([
    Input(shape=(X_train.shape[1],)),  # Define the input shape here
    Dense(units=64, activation='relu'), # HL1
    Dense(units=32, activation='relu'), # HL2
    Dense(units=1, activation='sigmoid') # Output layer
])

In [64]:
model_1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 64)                  │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

#### Compiling the model

In [65]:
# Adam Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

# Binary Cross-entropy loss
loss = tf.keras.losses.BinaryCrossentropy()

In [66]:
opt

In [67]:
loss

In [68]:
model_1.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

#### Training the model

In [69]:
# Setting up early-stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [70]:
# Setting up TensorBoard for model evaluation
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

tensor_board = TensorBoard(log_dir=log_dir, histogram_freq=1, write_images=True)

# Fitting data to the model
history = model_1.fit(x=X_train, y=y_train, epochs=100, callbacks=[early_stopping, tensor_board],
                   validation_data=(X_test, y_test))

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7911 - loss: 0.4579 - val_accuracy: 0.8485 - val_loss: 0.3690
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8460 - loss: 0.3708 - val_accuracy: 0.8505 - val_loss: 0.3634
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8495 - loss: 0.3599 - val_accuracy: 0.8400 - val_loss: 0.3649
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8422 - loss: 0.3685 - val_accuracy: 0.8520 - val_loss: 0.3542
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8496 - loss: 0.3569 - val_accuracy: 0.8490 - val_loss: 0.3484
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8546 - loss: 0.3519 - val_accuracy: 0.8525 - val_loss: 0.3503
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8528 - loss: 0.3508 - val_accuracy: 0.8470 - val_loss: 0.3534
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8574 - loss: 0.3465 - val_accu

In [71]:
model_1.save('Models/Churn_prediction.keras')

## Model Evaluation

In [72]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [73]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 2544), started 3:30:43 ago. (Use '!kill 2544' to kill it.)

## Hyperparameter tuning

## Predicting for new data

In [74]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [79]:
data = pd.DataFrame(data=[input_data], index=None)

In [80]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [82]:
new_data = preprocessor.transform(data)

In [83]:
new_data

array([[-0.53598516,  0.10479359, -0.69539349, -0.25781119,  0.80843615,
        -0.87683221,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ]])

In [85]:
model_1.predict(new_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([0.05117212], dtype=float32)

In [86]:
prediction_proba = model_1.predict(new_data)[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [87]:
print("Customer is likely to Churn") if prediction_proba > 0.5 else print("Customer is NOT likely to churn")

Customer is NOT likely to churn
